In [6]:
import pandas as pd  
import matplotlib.pyplot as plt  
from sklearn.ensemble import RandomForestClassifier  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  
  
# Load the dataset into a pandas DataFrame  
df = pd.read_csv('../data/preprocessed/main-data.csv')  
X = df[["item_count","cold_item","frozen_item","scalable_item"]]
Y= pd.cut(df['collection_duration'], bins=[0, 3, 9, float('inf')], labels=[0,1,2])  


X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

length_Train = len(X_train)
length_Test = len(X_test)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_standard = sc.transform(X_train)
X_test_standard = sc.transform(X_test)

print("X_train without standardising features")
print("--------------------------------------")
print(X_train)
print("")
print("X_train standardising features")
print("--------------------------------------")
print(X_train_standard)

X_train without standardising features
--------------------------------------
         item_count  cold_item  frozen_item  scalable_item
749391            4          0            0              0
1154737           8          4            1              0
1292598           5          1            1              1
1325600           4          0            0              0
1189916           4          1            0              0
...             ...        ...          ...            ...
110268            5          0            0              0
259178            8          0            3              0
131932            9          1            0              0
671155            5          4            0              0
121958            7          2            0              3

[1103845 rows x 4 columns]

X_train standardising features
--------------------------------------
[[-0.63745957 -0.83692167 -0.46273673 -0.54547169]
 [ 0.36737328  2.35632221  1.10352607 -0.54547169]
 [-0.38625136

In [8]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
# Fit the XGBoost classifier
xgb = XGBClassifier(objective='multi:softmax',
                    num_class=3,
                    multi_strategy='multi_output_tree',
                    subsample= 1.0, 
                    n_estimators= 200, 
                    min_child_weight= 5, 
                    max_depth= 3, 
                    learning_rate= 0.1, 
                    gamma= 0.2, 
                    colsample_bytree= 0.9,
                    )
xgb.fit(X_train, Y_train)

# Make predictions on the test set
preds = xgb.predict(X_test)

# Calculate accuracy
acc_xgb = accuracy_score(Y_test, preds)
print("XGBoost's prediction accuracy is: %3.2f" % (acc_xgb))
# Calculate precision
precision_xgb = precision_score(Y_test, preds, average='weighted')
print("XGBoost's precision score is: %3.2f" % (precision_xgb))

# Calculate recall
recall_xgb = recall_score(Y_test, preds, average='weighted')
print("XGBoost's recall score is: %3.2f" % (recall_xgb))

# Calculate f1 score
f1_xgb = f1_score(Y_test, preds, average='weighted')
print("XGBoost's f1 score is: %3.2f" % (f1_xgb))


XGBoost's prediction accuracy is: 0.81
XGBoost's precision score is: 0.72
XGBoost's recall score is: 0.81
XGBoost's f1 score is: 0.74


c:\Users\Amet\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [23]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Define the parameter grid to search
param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Initialize XGBoost classifier
#xgb = XGBClassifier(objective='multi:softprob','subsample': 1.0, 'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 3,  'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.9)

# Initialize GridSearchCV or RandomizedSearchCV
# For GridSearchCV
#grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='accuracy',n_jobs=-1)

# For RandomizedSearchCV
#grid_search = RandomizedSearchCV(estimator=xgb, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy',n_jobs=-1, random_state=42)

# Fit the GridSearchCV or RandomizedSearchCV
#grid_search.fit(X_train, Y_train)

# Get the best parameters
#best_params = grid_search.best_params_
#print("Best parameters:", best_params)

# Use the best parameters to fit the XGBoost classifier
xgb = XGBClassifier(objective='multi:softprob',
                    subsample= 1.0, 
                    n_estimators= 200, 
                    min_child_weight= 5, 
                    max_depth= 3, 
                    learning_rate= 0.1, 
                    gamma= 0.2, 
                    colsample_bytree= 0.9)
xgb.fit(X_train, Y_train)

# Make predictions on the test set
preds_best = xgb.predict(X_test)

# Calculate evaluation metrics using the best model
acc_xgb_best = accuracy_score(Y_test, preds_best)
precision_xgb_best = precision_score(Y_test, preds_best, average='weighted')
recall_xgb_best = recall_score(Y_test, preds_best, average='weighted')
f1_xgb_best = f1_score(Y_test, preds_best, average='weighted')

# Print the evaluation metrics
print("Best XGBoost's prediction accuracy is: %3.2f" % (acc_xgb_best))
print("Best XGBoost's precision score is: %3.2f" % (precision_xgb_best))
print("Best XGBoost's recall score is: %3.2f" % (recall_xgb_best))
print("Best XGBoost's f1 score is: %3.2f" % (f1_xgb_best))

Best XGBoost's prediction accuracy is: 0.76
Best XGBoost's precision score is: 0.68
Best XGBoost's recall score is: 0.76
Best XGBoost's f1 score is: 0.70


c:\Users\Amet\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

xgb = XGBClassifier(n_estimators=100)
xgb.fit(X_train, Y_train)

preds = xgb.predict(X_test)

# Accuracy
acc_xgb = (preds == Y_test).sum().astype(float) / len(preds)*100
print("XGBoost's prediction accuracy is: %3.2f" % (acc_xgb))

# Classification report
print("Classification Report:")
print(classification_report(Y_test, preds))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_test, preds))

XGBoost's prediction accuracy is: 84.05
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3696
           1       0.86      0.97      0.91    226373
           2       0.64      0.28      0.39     45893

    accuracy                           0.84    275962
   macro avg       0.50      0.42      0.43    275962
weighted avg       0.81      0.84      0.81    275962

Confusion Matrix:
[[     0   3696      0]
 [     0 218969   7404]
 [     0  32917  12976]]


c:\Users\Amet\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Amet\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Amet\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [ ]:
from sklearn.model_selection import RandomizedSearchCV  
from scipy.stats import randint  
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer  
  
# Define the parameter distributions to search  
param_dist = {  
    'n_estimators': randint(50, 150),  # Number of boosting stages  
    'learning_rate': [0.01, 0.1, 0.5],  # Step size shrinkage used in update to prevent overfitting  
    'max_depth': [3, 4, 5],  # Maximum depth of the individual estimators  
}  
  
# Define the scoring metrics  
scoring = {  
    'accuracy': make_scorer(accuracy_score),  
    'precision': make_scorer(precision_score),  
    'recall': make_scorer(recall_score),  
    'f1': make_scorer(f1_score)  
}  
  
# Initialize RandomizedSearchCV  
random_search = RandomizedSearchCV(estimator=xg, param_distributions=param_dist, n_iter=10, cv=5, scoring=scoring, refit='accuracy', random_state=10, n_jobs=-1)  
  
# Perform randomized search to find the best parameters  
random_search.fit(X_train_standard, Y_train)  
  
# Get the best parameters  
best_params = random_search.best_params_  
print("Best Parameters:", best_params)  
  
# Get the scores for all tried parameter combinations  
results = random_search.cv_results_  
tried_params = results['params']  
accuracy_scores = results['mean_test_accuracy']  
precision_scores = results['mean_test_precision']  
recall_scores = results['mean_test_recall']  
f1_scores = results['mean_test_f1']  
  
# Print the tried parameters and corresponding scores  
print("Tried Parameters and Scores:")  
for params, acc, prec, rec, f1 in zip(tried_params, accuracy_scores, precision_scores, recall_scores, f1_scores):  
    print("Parameters:", params)  
    print("Accuracy:", acc)  
    print("Precision:", prec)  
    print("Recall:", rec)  
    print("F1 Score:", f1)  
    print()  
  
# Train the model with the best parameters  
best_xg = GradientBoostingClassifier(**best_params, random_state=10)  
best_xg.fit(X_train_standard, Y_train)  
  
# Make predictions on the test set  
Y_pred_xg = best_xg.predict(X_test_standard)  
  
# Evaluate the model  
accuracy = accuracy_score(Y_test, Y_pred_xg)  
precision = precision_score(Y_test, Y_pred_xg)  
recall = recall_score(Y_test, Y_pred_xg)  
f1 = f1_score(Y_test, Y_pred_xg)  
  
print("Evaluation Metrics on Test Set:")  
print("Accuracy: %.2f" % accuracy)  
print("Precision: %.2f" % precision)  
print("Recall: %.2f" % recall)  
print("F1 Score: %.2f" % f1)  